## Testing Playground

You've probably noticed this project lacks a unit testing set-up. Honestly, I found unit testing tricky for this and felt visual inspections of outputs, like markdowns or visualizations, were generally more trustworthy.

But I get the worry of accidentally breaking things when contributing code. So, I made this notebook to help with that. It's a work in progress, aimed at letting you easily see specific outputs of interest for smoother development.

Previously, my basic testing meant using a `test.py` file to generate a few markdowns and then checking them manually. Or, for a deeper look, I'd run the `main.py` and wait a while to see everything, which isn't quick on my laptop.

This notebook aims to streamline that process, letting you test and inspect targeted parts of the output without the fear of breaking things.

**Before you begin, it's recommended that you put the `conversations.json` file close by, like in a `./data/` folder.**

#### Examples ...

In [ ]:
import json
from pathlib import Path
from typing import Callable, List

import controllers.file_system as fs_module
from models.conversation import Conversation
from models.conversation_set import ConversationSet

# function type
AttrFunc = Callable[[Conversation], int]

# Paths
conversations_path = Path("data") / "conversations.json"  # adjust path if needed
output_path = Path("output")
output_path.mkdir(exist_ok=True)

# Load conversations
with open(conversations_path, "r", encoding="utf-8") as f:
    conversations = json.load(f)

conversation_set = ConversationSet(conversations)


def clear_output():
    """Clear output folder"""
    for file in output_path.glob("*"):
        file.unlink()


# Utility function to get statistics and print conversations based on a criteria
def get_top_convos(attr_func: AttrFunc, description: str, count: int = 5):
    """Get statistics and save top conversations based on a criteria"""

    stats: List[int] = [attr_func(c) for c in conversation_set.conversation_list]
    avg_stat = sum(stats) / len(stats)
    median_stat = sorted(stats)[len(stats) // 2]
    max_stat = max(stats)

    print(
        f"Average {description}: {avg_stat}\n"
        f"Median {description}: {median_stat}\n"
        f"Max {description}: {max_stat}\n"
    )

    convos_sorted_by_attr = sorted(
        conversation_set.conversation_list, key=attr_func, reverse=True
    )

    for convo in convos_sorted_by_attr[:count]:
        print(
            f"id: {convo.id}\n"
            f"title: {convo.title}\n"
            f"{description}: {attr_func(convo)}\n"
        )
        file_path = output_path / f"{convo.file_name()}.md"
        fs_module.save_conversation_to_file(convo, file_path)
        print(f"saved to '{file_path.resolve()}'\n")

In [ ]:
clear_output()

In [ ]:
get_top_convos(lambda c: c.leaf_count(), "leaf count")

In [ ]:
get_top_convos(lambda c: c.message_count(), "message count")

In [ ]:
get_top_convos(lambda c: len(c.content_types()), "content type count")

In [ ]:
get_top_convos(lambda c: len(c.used_plugins()), "plugin count")